# CIFAR-10 Challange - 김영인

1. wide resnet 50-2
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
4. \+ regularization(dropout, weight decay)
5. \+ Optimizer(momentum, scheduler)


In [6]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [7]:
learning_rate = 0.1
scheduler_step = 60
scheduler_gamma = 0.2
training_epochs = 200
batch_size = 64
momentum = 0.9
weight_decay = 5e-4
dropout_rate = 0.3

## Load & Preprocess Data

In [8]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transforms.ToTensor())
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [9]:
print(models.wide_resnet50_2(pretrained=True))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [10]:
class WideResNet(nn.Module):
    def __init__ (self):
        super(WideResNet, self).__init__()
        self.resnet = models.wide_resnet50_2(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(1000, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        return x

## Load Model

In [11]:
net = WideResNet()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [12]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        
print('Finished Training')

[1,   100] loss: 2.482
[1,   200] loss: 2.171
[1,   300] loss: 2.004
[1,   400] loss: 1.910
[1,   500] loss: 1.827
[1,   600] loss: 1.807
[1,   700] loss: 1.762
[2,   100] loss: 1.706
[2,   200] loss: 1.690
[2,   300] loss: 1.642
[2,   400] loss: 1.622
[2,   500] loss: 1.611
[2,   600] loss: 1.598
[2,   700] loss: 1.545
[3,   100] loss: 1.505
[3,   200] loss: 1.491
[3,   300] loss: 1.459
[3,   400] loss: 1.397
[3,   500] loss: 1.377
[3,   600] loss: 1.338
[3,   700] loss: 1.369
[4,   100] loss: 1.299
[4,   200] loss: 1.269
[4,   300] loss: 1.268
[4,   400] loss: 1.242
[4,   500] loss: 1.271
[4,   600] loss: 1.237
[4,   700] loss: 1.231
[5,   100] loss: 1.205
[5,   200] loss: 1.168
[5,   300] loss: 1.176
[5,   400] loss: 1.187
[5,   500] loss: 1.148
[5,   600] loss: 1.184
[5,   700] loss: 1.137
[6,   100] loss: 1.119
[6,   200] loss: 1.125
[6,   300] loss: 1.116
[6,   400] loss: 1.109
[6,   500] loss: 1.114
[6,   600] loss: 1.135
[6,   700] loss: 1.096
[7,   100] loss: 1.094
[7,   200] 

[50,   200] loss: 0.786
[50,   300] loss: 0.837
[50,   400] loss: 0.814
[50,   500] loss: 0.807
[50,   600] loss: 0.826
[50,   700] loss: 0.828
[51,   100] loss: 0.786
[51,   200] loss: 0.800
[51,   300] loss: 0.797
[51,   400] loss: 0.814
[51,   500] loss: 0.834
[51,   600] loss: 0.852
[51,   700] loss: 0.846
[52,   100] loss: 0.763
[52,   200] loss: 0.811
[52,   300] loss: 0.795
[52,   400] loss: 0.804
[52,   500] loss: 0.824
[52,   600] loss: 0.836
[52,   700] loss: 0.844
[53,   100] loss: 0.778
[53,   200] loss: 0.789
[53,   300] loss: 0.772
[53,   400] loss: 0.782
[53,   500] loss: 0.811
[53,   600] loss: 0.837
[53,   700] loss: 0.855
[54,   100] loss: 0.790
[54,   200] loss: 0.815
[54,   300] loss: 0.821
[54,   400] loss: 0.807
[54,   500] loss: 0.832
[54,   600] loss: 0.797
[54,   700] loss: 0.833
[55,   100] loss: 0.773
[55,   200] loss: 0.794
[55,   300] loss: 0.812
[55,   400] loss: 0.826
[55,   500] loss: 0.838
[55,   600] loss: 0.813
[55,   700] loss: 0.820
[56,   100] loss

KeyboardInterrupt: 

## Test

In [13]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 40.600000 %
Accuracy of   car : 52.300000 %
Accuracy of  bird : 58.400000 %
Accuracy of   cat : 34.700000 %
Accuracy of  deer : 52.200000 %
Accuracy of   dog : 20.800000 %
Accuracy of  frog : 93.500000 %
Accuracy of horse : 57.300000 %
Accuracy of  ship : 56.800000 %
Accuracy of truck : 58.000000 %
Mean Accuracy :  52.46
